
 Redoing  the sympy code from
 
 [1] Coordinate Measuring Machine Error Compensation via Ball Plate Measurements - EF Howick Callaghan Innnovation Report 118
 `/home/elfnor/Documents/MSLNZ/CMM/Ballplate_MSL-_ipynb_format4/ipy_notebooks/Model_3D_CMM-unicode.ipynb  
 in 2024

In [29]:
import sympy 

In [30]:
#symbols used in model
xyz_terms = sympy.var('x,y,z')
prb_terms = sympy.var('xt,yt,zt')
Tterms = sympy.var('T(x:z)(x:z)')
Rterms = sympy.var('R(x:z)(x:z)')
terms = Tterms+Rterms

In [31]:
#Matrix expression
X = sympy.Matrix([[x + Txx],[Txy],[Txz]])
Y = sympy.Matrix([[Tyx],[y + Tyy],[Tyz]])
Z = sympy.Matrix([[Tzx],[Tzy],[z + Tzz]])
T = sympy.Matrix([[xt],[yt],[zt]])

RX = sympy.Matrix([[1,Rxz,-Rxy],[-Rxz,1,Rxx],[Rxy,-Rxx,1]])
RY_inv = sympy.Matrix([[1,-Ryz,Ryy],[Ryz,1,-Ryx],[-Ryy,Ryx,1]])
RZ_inv = sympy.Matrix([[1,-Rzz,Rzy],[Rzz,1,-Rzx],[-Rzy,Rzx,1]])

X, Y, Z, T, RX as defined in equation (2)  of [1] (pg.5)

In [26]:
X

Matrix([
[Txx + x],
[    Txy],
[    Txz]])

In [6]:
Y

Matrix([
[    Tyx],
[Tyy + y],
[    Tyz]])

In [7]:
Z

Matrix([
[    Tzx],
[    Tzy],
[Tzz + z]])

In [8]:
T

Matrix([
[xt],
[yt],
[zt]])

In [32]:
W_expr = (RX*Y - RX*X + RX*RY_inv*Z + RX*RY_inv*RZ_inv*T).expand()
W_expr

Matrix([
[    Rxy*Ryx*Rzx*zt - Rxy*Ryx*Rzz*xt - Rxy*Ryx*Tzy - Rxy*Ryx*yt + Rxy*Ryy*Rzy*zt - Rxy*Ryy*Rzz*yt + Rxy*Ryy*Tzx + Rxy*Ryy*xt - Rxy*Rzx*yt + Rxy*Rzy*xt + Rxy*Txz - Rxy*Tyz - Rxy*Tzz - Rxy*z - Rxy*zt - Rxz*Ryx*Rzx*yt + Rxz*Ryx*Rzy*xt - Rxz*Ryx*Tzz - Rxz*Ryx*z - Rxz*Ryx*zt + Rxz*Ryz*Rzy*zt - Rxz*Ryz*Rzz*yt + Rxz*Ryz*Tzx + Rxz*Ryz*xt - Rxz*Rzx*zt + Rxz*Rzz*xt - Rxz*Txy + Rxz*Tyy + Rxz*Tzy + Rxz*y + Rxz*yt + Ryy*Rzx*yt - Ryy*Rzy*xt + Ryy*Tzz + Ryy*z + Ryy*zt + Ryz*Rzx*zt - Ryz*Rzz*xt - Ryz*Tzy - Ryz*yt + Rzy*zt - Rzz*yt - Txx + Tyx + Tzx - x + xt],
[   -Rxx*Ryx*Rzx*zt + Rxx*Ryx*Rzz*xt + Rxx*Ryx*Tzy + Rxx*Ryx*yt - Rxx*Ryy*Rzy*zt + Rxx*Ryy*Rzz*yt - Rxx*Ryy*Tzx - Rxx*Ryy*xt + Rxx*Rzx*yt - Rxx*Rzy*xt - Rxx*Txz + Rxx*Tyz + Rxx*Tzz + Rxx*z + Rxx*zt - Rxz*Ryy*Rzx*yt + Rxz*Ryy*Rzy*xt - Rxz*Ryy*Tzz - Rxz*Ryy*z - Rxz*Ryy*zt - Rxz*Ryz*Rzx*zt + Rxz*Ryz*Rzz*xt + Rxz*Ryz*Tzy + Rxz*Ryz*yt - Rxz*Rzy*zt + Rxz*Rzz*yt + Rxz*Txx - Rxz*Tyx - Rxz*Tzx + Rxz*x - Rxz*xt - Ryx*Rzx*yt + Ryx*Rzy*xt - Ryx*Tzz 

In [33]:
#remove 2nd order terms
import itertools
remove_2nd_order = [(pair[0]*pair[1], 0) for pair in itertools.combinations_with_replacement(terms, 2)]
model = W_expr.subs(remove_2nd_order)
model

Matrix([
[-Rxy*z - Rxy*zt + Rxz*y + Rxz*yt + Ryy*z + Ryy*zt - Ryz*yt + Rzy*zt - Rzz*yt - Txx + Tyx + Tzx - x + xt],
[ Rxx*z + Rxx*zt + Rxz*x - Rxz*xt - Ryx*z - Ryx*zt + Ryz*xt - Rzx*zt + Rzz*xt - Txy + Tyy + Tzy + y + yt],
[        -Rxx*y - Rxx*yt - Rxy*x + Rxy*xt + Ryx*yt - Ryy*xt + Rzx*yt - Rzy*xt - Txz + Tyz + Tzz + z + zt]])

In [34]:
for row in model:
    print(row)

-Rxy*z - Rxy*zt + Rxz*y + Rxz*yt + Ryy*z + Ryy*zt - Ryz*yt + Rzy*zt - Rzz*yt - Txx + Tyx + Tzx - x + xt
Rxx*z + Rxx*zt + Rxz*x - Rxz*xt - Ryx*z - Ryx*zt + Ryz*xt - Rzx*zt + Rzz*xt - Txy + Tyy + Tzy + y + yt
-Rxx*y - Rxx*yt - Rxy*x + Rxy*xt + Ryx*yt - Ryy*xt + Rzx*yt - Rzy*xt - Txz + Tyz + Tzz + z + zt


In [35]:
#linear model Rxx -> RxxL*x  etc.
TtermsL = sympy.var('T(x:z)(x:z)L')
RtermsL = sympy.var('R(x:z)(x:z)L')
termsL = TtermsL + RtermsL 
dep_var = ([x]*3 + [y]*3 + [z]*3)*2

replace_with_linear = [(t[0] ,t[2]*t[1]) for t in zip(terms,termsL,dep_var)]
model_linear = model.subs(replace_with_linear)
model_linear

Matrix([
[-RxyL*x*z - RxyL*x*zt + RxzL*x*y + RxzL*x*yt + RyyL*y*z + RyyL*y*zt - RyzL*y*yt + RzyL*z*zt - RzzL*yt*z - TxxL*x + TyxL*y + TzxL*z - x + xt],
[RxxL*x*z + RxxL*x*zt + RxzL*x**2 - RxzL*x*xt - RyxL*y*z - RyxL*y*zt + RyzL*xt*y - RzxL*z*zt + RzzL*xt*z - TxyL*x + TyyL*y + TzyL*z + y + yt],
[          -RxxL*x*y - RxxL*x*yt - RxyL*x**2 + RxyL*x*xt + RyxL*y*yt - RyyL*xt*y + RzxL*yt*z - RzyL*xt*z - TxzL*x + TyzL*y + TzzL*z + z + zt]])

In [36]:
for row in model_linear:
    print(row)

-RxyL*x*z - RxyL*x*zt + RxzL*x*y + RxzL*x*yt + RyyL*y*z + RyyL*y*zt - RyzL*y*yt + RzyL*z*zt - RzzL*yt*z - TxxL*x + TyxL*y + TzxL*z - x + xt
RxxL*x*z + RxxL*x*zt + RxzL*x**2 - RxzL*x*xt - RyxL*y*z - RyxL*y*zt + RyzL*xt*y - RzxL*z*zt + RzzL*xt*z - TxyL*x + TyyL*y + TzyL*z + y + yt
-RxxL*x*y - RxxL*x*yt - RxyL*x**2 + RxyL*x*xt + RyxL*y*yt - RyyL*xt*y + RzxL*yt*z - RzyL*xt*z - TxzL*x + TyzL*y + TzzL*z + z + zt


For a linear model TxyL is equivalent to TyxL in the workpiece CSY see Figure 4 of [1]
Is this true for other machine configs?
I have implemented sliders for a 21 parameter model and use both TxyL and TyxL in `cmm_error_map.design_matrix_linear_fixed.model_linear`
so ignore this 

What did I do with the squareness terms?
```
    xE = (
        x * TxxL
        - y * TyxL
        - z * TzxL
        - (z + zt) * x * RxyL
        + (y + yt) * x * RxzL
        - (z + zt) * y * RyyL
        + yt * y * RyzL
        - zt * z * RzyL
        + yt * z * RzzL
        - (y + yt) * Wxy
        - (z + zt) * Wxz
    )
```


In [37]:
for row in model_linear:
    print(row.collect(terms))

-RxyL*x*z - RxyL*x*zt + RxzL*x*y + RxzL*x*yt + RyyL*y*z + RyyL*y*zt - RyzL*y*yt + RzyL*z*zt - RzzL*yt*z - TxxL*x + TyxL*y + TzxL*z - x + xt
RxxL*x*z + RxxL*x*zt + RxzL*x**2 - RxzL*x*xt - RyxL*y*z - RyxL*y*zt + RyzL*xt*y - RzxL*z*zt + RzzL*xt*z - TxyL*x + TyyL*y + TzyL*z + y + yt
-RxxL*x*y - RxxL*x*yt - RxyL*x**2 + RxyL*x*xt + RyxL*y*yt - RyyL*xt*y + RzxL*yt*z - RzyL*xt*z - TxzL*x + TyzL*y + TzzL*z + z + zt


There are sign differences between these two?

In [31]:
for row in model_linear:
    print(row.collect(terms).collect(xyz_terms))
    

x*(-RxyL*z - RxyL*zt + RxzL*y + RxzL*yt - TxxL - 1) + xt + y*(RyyL*z + RyyL*zt - RyzL*yt + TyxL) + z*(RzyL*zt - RzzL*yt + TzxL)
RxzL*x**2 + x*(RxxL*z + RxxL*zt - RxzL*xt - TxyL) + y*(-RyxL*z - RyxL*zt + RyzL*xt + TyyL + 1) + yt + z*(-RzxL*zt + RzzL*xt + TzyL)
-RxyL*x**2 + x*(-RxxL*y - RxxL*yt + RxyL*xt - TxzL) + y*(RyxL*yt - RyyL*xt + TyzL) + z*(RzxL*yt - RzyL*xt + TzzL + 1) + zt


In [32]:
for row in model_linear:
    print(row.collect(xyz_terms).collect(terms))

x*(-RxyL*z - RxyL*zt + RxzL*y + RxzL*yt - TxxL - 1) + xt + y*(RyyL*z + RyyL*zt - RyzL*yt + TyxL) + z*(RzyL*zt - RzzL*yt + TzxL)
RxzL*x**2 + x*(RxxL*z + RxxL*zt - RxzL*xt - TxyL) + y*(-RyxL*z - RyxL*zt + RyzL*xt + TyyL + 1) + yt + z*(-RzxL*zt + RzzL*xt + TzyL)
-RxyL*x**2 + x*(-RxxL*y - RxxL*yt + RxyL*xt - TxzL) + y*(RyxL*yt - RyyL*xt + TyzL) + z*(RzxL*yt - RzyL*xt + TzzL + 1) + zt


In [21]:
model_linear.collect(termsL)

AttributeError: 'MutableDenseMatrix' object has no attribute 'collect'

In [23]:
Rterms = sympy.var('R_(x:z)(x:z)')
Tterms = sympy.var('T_(x:z)(x:z)')

RX = sympy.Matrix([[1,0,-R_xy],[0,1,0],[R_xy,0,1]])
RY_inv = sympy.Matrix([[1,0,0],[0,1,0],[0,0,1]])
RZ_inv = sympy.Matrix([[1,0,0],[0,1,0],[0,0,1]])

In [25]:
X = sympy.Matrix([[x],[0],[0]])
Y = sympy.Matrix([[0],[y],[0]])
Z = sympy.Matrix([[0],[0],[z]])
T = sympy.Matrix([[0],[0],[0]])



W_expr = (RX*Y - RX*X + RX*RY_inv*Z + RX*RY_inv*RZ_inv*T)
W_expr

Matrix([
[-R_xy*z - x],
[          y],
[-R_xy*x + z]])

In [21]:
RX*Y

Matrix([
[0],
[y],
[0]])

In [22]:
RX*X

Matrix([
[     x],
[     0],
[R_xy*x]])

In [26]:
RX*RY_inv*Z

Matrix([
[-R_xy*z],
[      0],
[      z]])

In [27]:
RX

Matrix([
[   1, 0, -R_xy],
[   0, 1,     0],
[R_xy, 0,     1]])

In [ ]:
RY

In [8]:
sympy.var('a1 a2 a3 b1 b2 b3 c1 c2 c3')
M = sympy.Matrix([[a1, b1, c1], [a2,b2,c2], [a3,b3,c3]])

In [12]:
M * Z

Matrix([
[c1*z],
[c2*z],
[c3*z]])

TypeError: unsupported operand type(s) for @: 'MutableDenseMatrix' and 'Symbol'